<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-11 12:30:22
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.41 C
-------------------
Today PnL: -56.13 K (-0.4%)
Current PnL: -28.79 L (-18.36%)
CY Booked + Current PnL: -14.05 L (-8.96%)
-------------------
Total profit:  1.64 L
Total loss:  -30.43 L
-------------------
Total Booked + Current PnL: 12.93 L (9.96%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.48%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 94.17 L (66.93%)
Deployed:  1.30 C
Current:  1.41 C
CAGR/XIRR %: 4.17%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-0.48,2.27,18.87,21.57,18908.0,2220.0,100200.0,397.04,-11.92,61.0,H-SC,3.44,168.0,0.12,0.71,46.22,XR,ATH,FINANCE
18,COALINDIA,-2.29,3.55,15.13,19.23,23574.0,5346.0,155807.0,484.83,21.35,48.0,L-LC,12.22,182.0,0.23,1.11,25.59,XY25,ATH,MINING
77,TTKPRESTIG,-0.09,-24.03,31.75,0.09,24309.0,-24214.0,76563.0,770.00,77.80,49.0,M-SC,8.47,253.0,-1.00,0.54,5.37,OX40N,NTT,DURABLES
5,ANGELONE,0.04,14.00,8.94,24.20,31111.0,42750.0,348000.0,3033.00,62.46,62.0,X-SC,3.08,99.0,1.37,2.47,45.75,X40N,NTT,FINANCE
36,ICICIGI,0.94,2.10,16.88,19.34,34488.0,4206.0,204315.0,2252.93,-19.32,60.0,X-MC,3.54,71.0,0.12,1.45,18.37,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,WHIRLPOOL,2.61,-27.36,150.43,81.92,125448.0,-31405.0,83393.0,2270.0,-57.51,64.0,M-SC,31.78,218.0,-0.25,0.59,17.14,XR,NTT,DURABLES
45,JCHAC,2.14,-36.43,57.28,-0.01,46540.0,-46555.0,81250.0,2282.0,16530.43,57.0,M-SC,17.03,235.0,-1.00,0.58,12.65,OX40N,BTT,AC
28,GLAXO,2.12,7.52,29.41,39.15,63017.0,14988.0,214272.0,3466.2,-13.65,70.0,X-MC,13.44,62.0,0.24,1.52,35.75,X40,ATH,PHARMA
68,STARHEALTH,1.96,-7.51,59.22,47.26,153975.0,-21118.0,260005.0,761.0,26.23,65.0,H-SC,1.04,174.0,-0.14,1.85,39.30,XY24,NTT,INSURANCE
75,TMPV,1.74,-20.90,55.46,22.98,99104.0,-47203.0,178695.0,600.0,-18.19,69.0,X-LC,3.56,2.0,-0.48,1.27,14.24,XY24,NTT,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,-3.22,-13.51,68.31,45.57,116464.0,-26639.0,170493.0,844.00,3694.59,69.0,M-SC,7.14,245.0,-0.23,1.21,29.02,XY24,NTT,AUTO
54,QUESS,-2.98,-29.04,101.30,42.84,46728.0,-18878.0,46128.0,424.00,-53.72,55.0,X-SC,20.29,83.0,-0.40,0.33,10.96,XY24,NTT,MISC
19,COFFEEDAY,-2.64,-46.36,152.21,35.27,92699.0,-52647.0,60902.0,80.00,-59.76,47.0,L-SC,14.17,268.0,-0.57,0.43,49.06,XR,NTT,HOTELS
76,TRIDENT,-2.55,-28.27,79.17,28.51,52388.0,-26084.0,66171.0,48.00,-14.99,50.0,M-SC,8.89,226.0,-0.50,0.47,13.23,XR,NTT,TEXTILES
18,COALINDIA,-2.29,3.55,15.13,19.23,23574.0,5346.0,155807.0,484.83,21.35,48.0,L-LC,12.22,182.0,0.23,1.11,25.59,XY25,ATH,MINING


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,-0.83,3.86,100.19,107.91,153631.0,5696.0,153340.0,1641.55,-15.69,68.0,M-SC,10.53,220.0,0.04,1.09,15.36,OX40N,ATH,CABLES
66,SIS,-0.01,-20.01,51.59,21.26,45820.0,-22216.0,88816.0,528.00,2101.04,68.0,H-SC,4.19,163.0,-0.48,0.63,22.44,OX40N,NTT,MISC
77,TTKPRESTIG,-0.09,-24.03,31.75,0.09,24309.0,-24214.0,76563.0,770.00,77.80,49.0,M-SC,8.47,253.0,-1.00,0.54,5.37,OX40N,NTT,DURABLES
72,TATAELXSI,-2.09,-24.07,73.88,32.03,73955.0,-31733.0,100102.0,9161.00,-17.88,45.0,H-SC,6.13,157.0,-0.43,0.71,12.12,OX40N,NTT,IT
70,SYMPHONY,-2.14,-28.26,39.34,-0.03,48302.0,-48361.0,122780.0,1306.00,-42.47,59.0,M-SC,10.85,206.0,-1.00,0.87,13.46,OX40N,NTT,DURABLES


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-2.29,3.55,15.13,19.23,23574.0,5346.0,155807.0,484.83,21.35,48.0,L-LC,12.22,182.0,0.23,1.11,25.59,XY25,ATH,MINING
50,MASFIN,-0.48,2.27,18.87,21.57,18908.0,2220.0,100200.0,397.04,-11.92,61.0,H-SC,3.44,168.0,0.12,0.71,46.22,XR,ATH,FINANCE
26,FINCABLES,-0.83,3.86,100.19,107.91,153631.0,5696.0,153340.0,1641.55,-15.69,68.0,M-SC,10.53,220.0,0.04,1.09,15.36,OX40N,ATH,CABLES
1,ABB,-1.03,6.59,36.57,45.57,101978.0,17237.0,278856.0,7934.00,-31.49,70.0,H-MC,3.40,121.0,0.17,1.98,23.83,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-1.03,6.59,36.57,45.57,101978.0,17237.0,278856.0,7934.00,-31.49,70.0,H-MC,3.40,121.0,0.17,1.98,23.83,AR,NTT,ELECTRICAL
50,MASFIN,-0.48,2.27,18.87,21.57,18908.0,2220.0,100200.0,397.04,-11.92,61.0,H-SC,3.44,168.0,0.12,0.71,46.22,XR,ATH,FINANCE
26,FINCABLES,-0.83,3.86,100.19,107.91,153631.0,5696.0,153340.0,1641.55,-15.69,68.0,M-SC,10.53,220.0,0.04,1.09,15.36,OX40N,ATH,CABLES
39,INDIAMART,-0.86,-4.11,115.58,106.72,136696.0,-5066.0,118270.0,4810.62,-52.54,51.0,H-SC,7.75,138.0,-0.04,0.84,18.31,AR,ATH,MISC
84,WIPRO,-0.59,-5.62,82.52,72.27,131971.0,-9522.0,159926.0,420.00,-18.78,36.0,M-LC,4.42,101.0,-0.07,1.14,2.80,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,-1.54,-19.55,50.58,21.14,92510.0,-44441.0,182898.0,680.00,-25.27,24.0,X-MC,10.10,75.0,-0.48,1.30,0.00,XY24,NTT,PAINTS
6,ASIANPAINT,-0.48,-16.74,45.25,20.95,94859.0,-42134.0,209634.0,3460.25,-17.64,28.0,X-LC,8.06,36.0,-0.44,1.49,12.94,X40,ATH,PAINTS
42,INFY,-1.00,0.94,32.99,34.24,104194.0,2948.0,315836.0,1972.00,-21.39,29.0,X-LC,5.23,6.0,0.03,2.25,9.30,X40,NTT,IT
33,HCLTECH,-1.25,0.82,22.83,23.83,55682.0,1976.0,243900.0,1908.19,5.13,31.0,X-LC,6.53,13.0,0.04,1.73,17.02,X40,ATH,IT
8,AWL,-0.03,-32.01,131.19,57.19,290897.0,-104390.0,221737.0,485.00,-67.84,34.0,X-MC,16.86,58.0,-0.36,1.58,1.05,XY24,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,-1.59,-19.24,46.80,18.55,142950.0,-72789.0,305448.0,4311.58,-30.42,36.0,X-LC,1.54,3.0,-0.51,2.17,3.92,X40,ATH,IT
81,VBL,-0.18,-7.95,47.39,35.67,137780.0,-25105.0,290737.0,671.64,-18.89,42.0,X-LC,1.63,5.0,-0.18,2.07,4.80,X40N,ATH,FMCG
21,DABUR,0.19,1.80,41.16,43.70,103945.0,4472.0,252540.0,735.00,-6.68,59.0,X-MC,1.94,73.0,0.04,1.80,17.12,XY24,BTT,FMCG
34,HINDUNILVR,0.51,-4.07,18.49,13.67,45596.0,-10463.0,246600.0,2922.00,-11.27,65.0,X-LC,2.51,9.0,-0.23,1.75,16.42,XY25,NTT,FMCG
82,VOLTAS,1.04,16.64,24.48,45.20,54750.0,31908.0,223650.0,1856.00,5.28,64.0,X-MC,2.95,80.0,0.58,1.59,24.57,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,-1.54,-19.55,50.58,21.14,92510.0,-44441.0,182898.0,680.0,-25.27,24.0,X-MC,10.10,75.0,-0.48,1.30,0.00,XY24,NTT,PAINTS
8,AWL,-0.03,-32.01,131.19,57.19,290897.0,-104390.0,221737.0,485.0,-67.84,34.0,X-MC,16.86,58.0,-0.36,1.58,1.05,XY24,NTT,FMCG
2,ABBOTINDIA,-1.21,-9.54,30.52,18.07,49378.0,-17065.0,161790.0,35195.0,-22.48,41.0,X-MC,9.78,64.0,-0.35,1.15,1.24,X40,ATH,PHARMA
44,ITC,-1.09,-22.09,42.18,10.78,78979.0,-53075.0,187243.0,452.0,-53.41,40.0,X-LC,18.85,1.0,-0.67,1.33,3.13,X40,NTT,FMCG
3,ACC,-0.40,-29.04,131.32,64.15,221747.0,-69091.0,168860.0,3906.0,-58.50,47.0,X-SC,7.65,84.0,-0.31,1.20,3.22,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-2.98,-29.04,101.30,42.84,46728.0,-18878.0,46128.0,424.00,-53.72,55.0,X-SC,20.29,83.0,-0.40,0.33,10.96,XY24,NTT,MISC
58,RELAXO,-0.87,-52.08,217.84,52.31,151530.0,-75600.0,69560.0,1176.00,-48.99,42.0,X-SC,13.16,92.0,-0.50,0.49,3.44,X40N,NTT,FOOTWEAR
12,BATAINDIA,-1.63,-41.71,131.96,35.20,98969.0,-53671.0,74999.0,2096.00,-1.66,54.0,X-SC,21.00,93.0,-0.54,0.53,6.58,X40,NTT,FOOTWEAR
51,MEDANTA,0.54,-4.36,29.74,24.08,36108.0,-5538.0,121412.0,1486.00,-12.07,53.0,X-SC,11.57,91.0,-0.15,0.86,11.20,XY24,NTT,HEALTHCARE
35,HONAUT,-0.05,-20.99,81.40,43.32,104746.0,-34192.0,128680.0,58357.33,-32.13,43.0,X-SC,9.86,90.0,-0.33,0.91,4.48,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-1.09,-22.09,42.18,10.78,78979.0,-53075.0,187243.0,452.00,-53.41,40.0,X-LC,18.85,1.0,-0.67,1.33,3.13,X40,NTT,FMCG
75,TMPV,1.74,-20.90,55.46,22.98,99104.0,-47203.0,178695.0,600.00,-18.19,69.0,X-LC,3.56,2.0,-0.48,1.27,14.24,XY24,NTT,AUTO
73,TCS,-1.59,-19.24,46.80,18.55,142950.0,-72789.0,305448.0,4311.58,-30.42,36.0,X-LC,1.54,3.0,-0.51,2.17,3.92,X40,ATH,IT
81,VBL,-0.18,-7.95,47.39,35.67,137780.0,-25105.0,290737.0,671.64,-18.89,42.0,X-LC,1.63,5.0,-0.18,2.07,4.80,X40N,ATH,FMCG
42,INFY,-1.00,0.94,32.99,34.24,104194.0,2948.0,315836.0,1972.00,-21.39,29.0,X-LC,5.23,6.0,0.03,2.25,9.30,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-1.19,-32.33,96.38,32.89,50355.0,-24960.0,52246.0,1800.00,-745.16,55.0,L-MC,11.53,259.0,-0.50,0.37,43.88,XR,NTT,BANKS
67,SONACOMS,0.57,-7.60,50.54,39.09,47239.0,-7691.0,93468.0,804.02,-27.69,68.0,M-SC,3.14,227.0,-0.16,0.66,32.23,AR,ATH,AUTO
50,MASFIN,-0.48,2.27,18.87,21.57,18908.0,2220.0,100200.0,397.04,-11.92,61.0,H-SC,3.44,168.0,0.12,0.71,46.22,XR,ATH,FINANCE
14,BSOFT,-0.01,-15.61,78.69,50.79,93397.0,-21963.0,118690.0,831.70,8.23,62.0,H-SC,7.70,171.0,-0.24,0.84,38.19,XR,ATH,IT
79,VAIBHAVGBL,-1.20,-24.09,111.79,60.77,155102.0,-44031.0,138744.0,521.00,59.78,57.0,H-SC,2.99,158.0,-0.28,0.99,25.79,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,0.04,14.00,8.94,24.20,31111.0,42750.0,348000.0,3033.00,62.46,62.0,X-SC,3.08,99.0,1.37,2.47,45.75,X40N,NTT,FINANCE
11,BANDHANBNK,1.72,-18.43,135.90,92.42,308778.0,-51349.0,227210.0,400.00,60.20,74.0,H-SC,8.56,169.0,-0.17,1.62,31.31,XY24,NTT,BANKS
67,SONACOMS,0.57,-7.60,50.54,39.09,47239.0,-7691.0,93468.0,804.02,-27.69,68.0,M-SC,3.14,227.0,-0.16,0.66,32.23,AR,ATH,AUTO
1,ABB,-1.03,6.59,36.57,45.57,101978.0,17237.0,278856.0,7934.00,-31.49,70.0,H-MC,3.40,121.0,0.17,1.98,23.83,AR,NTT,ELECTRICAL
27,GILLETTE,0.44,4.45,24.20,29.72,64175.0,11289.0,265185.0,10978.86,-14.09,69.0,X-SC,11.15,87.0,0.18,1.89,22.71,X40,ATH,FMCG


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.38
1,25,44.67
2,50,76.46


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.83
MC    30.68
LC    23.46
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.67
X40      23.24
X40N     15.17
XR        8.89
AR        8.87
XY25      8.72
OX40N     7.51
SR        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.10
H-SC    22.99
X-LC    20.10
M-SC    11.63
X-SC    10.43
H-MC     4.83
M-MC     1.38
M-LC     1.14
L-LC     1.11
H-LC     1.11
L-SC     0.78
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.03,-12.12,49.80
FINANCE,13.43,-10.52,54.01
IT,12.10,-25.58,88.32
MISC,7.15,-29.33,82.24
ELECTRICAL,6.06,-9.86,49.95
PAINTS,5.01,-31.02,50.48
INSURANCE,4.79,-0.72,35.19
PHARMA,4.07,-2.47,34.72
AUTO,3.14,-18.42,59.37


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3221658.0,21
AR,1321462.0,10
XR,1239382.0,12
X40,1226830.0,15
X40N,984675.0,10
OX40N,735047.0,10
XY25,393037.0,6
SR,294740.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3561493.0,24
X-MC,1635810.0,16
M-SC,1419775.0,15
X-LC,1063378.0,12
X-SC,822772.0,9
H-MC,398505.0,3
L-SC,180944.0,2
M-LC,131971.0,1
H-LC,77234.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1220503.0      6
           AR         935117.0      5
M-SC       XY24       799357.0      6
H-SC       XR         786521.0      7
X-LC       X40        532715.0      6
X-MC       X40        530971.0      7
           XY24       487352.0      3
           X40N       407434.0      4
X-SC       X40N       355045.0      4
H-SC       OX40N      324612.0      4
M-SC       OX40N      322190.0      5
X-SC       XY24       304583.0      3
H-SC       SR         294740.0      2
X-LC       X40N       222196.0      2
X-MC       XY25       210053.0      2
H-MC       XY24       209786.0      1
X-LC       XY24       200077.0      2
H-MC       AR         188719.0      2
M-SC       XR         177836.0      2
X-SC       X40        163144.0      2
M-LC       XR         131971.0      1
M-SC       AR         120392.0      2
X-LC       XY25       108390.0      2
L-SC       XR          92699.0      1
           OX40N       88245.0      1
H-LC       AR          77234.0      1
M-MC       XY25        51020.0      1
L-MC       XR          50355.0      1
L-LC       XY25        23574.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 23.0 seconds
